In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [7]:
print(y_test[0])

7


In [8]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras import Model 

In [ ]:
class MNISTModel(Model):
    def __init__(self):
        super(MNISTModel, self).__init()
        self.conv1 = Conv2D(filters=32, kernel_size=3, activation='relu')
        self.flatten = Flatten()
        self.dense1 = Dense(neurons=128, activation='relu')
        self.dense2 = Dense(neurons=10, activation='softmax')
    
    def call(self, x):
        x1 = self.conv1(x)
        x2 = self.flatten(x1)
        x3 = self. dense1(x2)
        return self.dense2(x3)
        
model = MNISTModel()
